In [456]:
import pandas as pd
import numpy as np

In [403]:
df = pd.read_csv('/Users/binhn/Documents/documents/Portfolio/data_cleaning/raw_data/Building_Permits.csv')
df.head()

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1063641727.py:1: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/binhn/Documents/documents/Portfolio/data_cleaning/raw_data/Building_Permits.csv')


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


Handling missing data

In [404]:
# Missing value of each column
cols_missing_cells = df.isnull().sum()
cols_missing_cells

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

In [405]:
# Percentage of missing value per column
shape = df.shape
num_rows = shape[0]
percentage_missing_value_per_col = (cols_missing_cells/num_rows)*100
percentage_missing_value_per_col

Permit Number                              0.000000
Permit Type                                0.000000
Permit Type Definition                     0.000000
Permit Creation Date                       0.000000
Block                                      0.000000
Lot                                        0.000000
Street Number                              0.000000
Street Number Suffix                      98.885872
Street Name                                0.000000
Street Suffix                              1.391654
Unit                                      85.178984
Unit Suffix                               99.014077
Description                                0.145802
Current Status                             0.000000
Current Status Date                        0.000000
Filed Date                                 0.000000
Issued Date                                7.511312
Completed Date                            51.135747
First Construction Document Date           7.514329
Structural N

In [406]:
# Percentage of missing value
num_cells = np.product(df.shape)
num_missing_cells = cols_missing_cells.sum()
missing_percent = (num_missing_cells/num_cells)*100
missing_percent

26.26002315058403

- It can be seen that the column Street Number Suffix has 98.89% of its data missing. According to the data dictionary, this column relates to address, so the reason for this missing might be the fact that there is not any street number suffix in the address. Similar explanation can be applied to Street Suffix


- It can be seen that the column Unit has 85.178984 of its data missing. According to the data dictionary, this column provides information about the unit of a building. Because 72.63% of the remaining cells in Unit equal 0, it is reasonable to fill the missing cells with 0.

In [407]:
# Percentage of Unit = 0 among the remaining cells
value_counts = df['Unit'].value_counts()
total_rows = df['Unit'].shape[0] - df['Unit'].isnull().sum()
percentage_0 = (value_counts[0]/total_rows)*100
percentage_0

72.62797245496795

- It can be seen that the column Unit Suffix has 99.01% of its data missing. According to the data dictionary, this column provides information about the suffix if any, for the unit, so it is reasonable for this column to contain many missing cells.


- It can be seen that the column Description has 0.146% of its data missing. According to the data dictionary, this column provides details about purpose of the permit, so an expanation for the missing cells is an error in data entry. Because the proportion is small, the missing rows should be drop.

In [408]:
# Drop rows that have Description equals NA
df_clean = df.dropna(subset='Description')

- It can be seen that the column Issued Date has 7.5% of its data missing. According to the data dictionary, this column provides details about the issued date for the permit. A propbable reason for the missing data is error in data entry. However, because the proportion of missing data is quite significant, we can't drop those missing cells. A proposed solution is to manually look at the hard copy of those permits and fill out the missing data.

- It can be seen that the column Completed Date has 51.14% of its data missing. According to the data dictionary, this column provides the date on which project was completed, applicable if Current Status = "complete". The percentage of cells in Current Status that doesn't equal "complete" almost equals to the percentage of missing data in Completed Date. In addition. there isn't any rows that have Current Status equals complete and Completed Date is null. So the missing data is reasonable.

In [409]:
# Percentage of Current Status that doesn't equal "complete"
percentage_current_status_complete = (df[df['Current Status']=='complete'].shape[0]/df.shape[0])*100
100 - percentage_current_status_complete

51.19306184012066

In [410]:
# Number of rows that have Current Status = 'complete' and Completed Date is null
df[(df['Current Status'] == 'complete') & (df['Completed Date'].isnull())].shape

(0, 43)

- It can be seen that the column First Construction Document Date has 7.5% of its data missing. According to the data dictionary, this column provides date on which construction was documented. A reason for these missing data can be the error in data entry. However, because the amount of missing data is significant, we can not remove those rows. A solution is to look at the physical document and manually update the data. 

- It can be seen that the column Structural Notification has 96.52% of its data missing. According to the data dictionary, this column provides notification to meet some legel need, given or not. In addition, checking the unique values of this column, there are only 2 values: nan and "Y". In conclusion, we can fill out nan with "N".

In [411]:
# Fill out missing value of Structural Notification with "N"
df_clean['Structural Notification'].fillna(value="N", inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/914587679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Structural Notification'].fillna(value="N", inplace=True)


- It can be seen that the column Number of Existing Stories has 21.51% of its data missing. According to the data dictionary, this column provides the number of existing stories in the building. However, this is not applicable for certain permit types. To be specific, permit types 1, 2 and 5 don't have this attribute. 
- There are 20.74% of the permit types 3, 4, 6, 7, 8 that have null Number of Existing Stories. This can only be solved by looking at the physical document and manually enter the data.
- In addition, there is 1 row of permit types 2 that have Number of Existing Stories. This row should be remove.

In [412]:
# Percentage of permit type 3, 4, 6, 7 and 8 that have null Number of Existing Stories
(df[(df['Number of Existing Stories'].isnull()) & (df['Permit Type'].isin([3, 4, 6, 7, 8]))].shape[0]/df['Permit Type'].isin([3, 4, 6, 7, 8]).shape[0])*100

20.811965811965813

In [413]:
# Drop row of permit types 2 that have Number of Existing Stories
df_clean.drop(df_clean[(df_clean['Number of Existing Stories'].notnull()) & (df_clean['Permit Type'].isin([1, 2, 5]))].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/2635227126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean['Number of Existing Stories'].notnull()) & (df_clean['Permit Type'].isin([1, 2, 5]))].index, inplace=True)


- It can be seen that the column Number of Proposed Stories has 21.55% of its data missing. According to the data dictionary, this column provides the number of proposed stories for the construction/alteration. According to the permit, permit types 4, 7, 5, 6 don't have this attribute.
- There are 19.42% of the permit types 1, 2, 3, 8 that have null Number of Proposed Stories. This can only be solved by looking at the physical document and manually enter the data 

In [414]:
# Percentage of permit type 1, 2, 3 and 8 that have null Number of Proposed Stories
(df[(df['Number of Proposed Stories'].isnull()) & (df['Permit Type'].isin([1, 2, 3, 8]))].shape[0]/df['Permit Type'].isin([1, 2, 3, 8]).shape[0])*100

19.49421820010055

- It can be seen that the column Voluntary Soft-Story Retrofit has 99.98% of its data missing. According to the data dictionary, this column provides information about soft story to meet earth quake regulations.
- The column has only two types of value: "nan" and "Y". It is reasonable to fill out the "nan" value with "N"

In [415]:
# Fill out missing value of Voluntary Soft-Story Retrofit with "N"
df_clean['Voluntary Soft-Story Retrofit'].fillna(value="N", inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/3799486737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Voluntary Soft-Story Retrofit'].fillna(value="N", inplace=True)


- It can be seen that the column Fire Only Permit has 90.53% of its data missing. According to the data dictionary, this column provides information about fire hazard prevention related permit.
- The column has only two types of value: "nan" and "Y". It is reasonable to fill out the "nan" value with "N"

In [416]:
# Fill out missing value of Fire Only Permit with "N"
df_clean['Fire Only Permit'] = df_clean['Fire Only Permit'].fillna(value="N", inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1487596098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Fire Only Permit'] = df_clean['Fire Only Permit'].fillna(value="N", inplace=True)
/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1487596098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Fire Only Permit'] = df_clean['Fire Only Permit'].fillna(value="N", inplace=True)


- It can be seen that the column Permit Expiration Date has 26.08% of its data missing. According to the data dictionary, this column provides expiration date related to issued permit.
- One explanation for these missing data is error in data entry. A solution for this is to look into the physical document and manually enter the data

- It can be seen that the column Estimated Cost has 19.14% of its data missing. According to the data dictionary, this column provides the initial estimation of the cost of the project.
- According to the permit, only permit type 6 doesn't have this attribute. However, there are 597 rows of Permit Type = 6 that have not null Estimated Cost. These rows should be dropped. 

In [417]:
# Drop rows that have Permit Type = 6 and Estimated Cost is not null
df_clean.drop(df_clean[(df_clean['Estimated Cost'].notnull())&(df_clean['Permit Type']==6)].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/3523202669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean['Estimated Cost'].notnull())&(df_clean['Permit Type']==6)].index, inplace=True)


- It can be seen that the column Revised Cost has 3.05% of its data missing. According to the data dictionary, this column provides revised estimation of the cost of the project.
- According to the permit, permit types 4, 5, 6 and 7 don't have this atrribute. However, 1.7% of these permit types have Revised Cost that are not null. This rows should be dropped.
- 2.63% of the permit types 1, 2, 3 and 8 have null Revised Cost. This can be explained by the possibility that the estimated cost is correct.

In [418]:
# Drop rows that have Permit Type of 4, 5, 6 or 7 and Revised Cost is not null
df_clean.drop(df_clean[(df_clean["Revised Cost"].notnull())&(df_clean["Permit Type"].isin([4, 5, 6, 7]))].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1938071347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean["Revised Cost"].notnull())&(df_clean["Permit Type"].isin([4, 5, 6, 7]))].index, inplace=True)


In [419]:
# Percentage of permit types 1, 2, 3 and 8 have null revised cost
(df[(df["Revised Cost"].isnull())&(df["Permit Type"].isin([1, 2, 3, 8]))].shape[0]/df["Permit Type"].isin([1, 2, 3, 8]).shape[0])*100

2.679738562091503

- It can be seen that the column Existing Use has 20.67% of its data missing. According to the data dictionary, this column provides existing use of the building.
- According to the permit, permit types 1, 2 and 5 don't have this attribute. However, there is 1 line of permit type 2 has this attribute, this line should be dropped.
- There are 19.9% of data among permit types 3, 4, 6, 7 and 8 that have null Existing Use. This might be the result of error in data entry. A solution for this is to look at the physical permit and manually enter the data  

In [420]:
# Drop row that has permit type 2 and Existing Use is not null
df_clean.drop(df_clean[(df_clean['Existing Use'].notnull())&(df_clean['Permit Type'].isin([1, 2, 5]))].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/2302710219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean['Existing Use'].notnull())&(df_clean['Permit Type'].isin([1, 2, 5]))].index, inplace=True)


In [421]:
# Percentage of data among permit types 3, 4, 6, 7 and 8 that have null Existing Use.
(df[(df['Existing Use'].isnull())&(df['Permit Type'].isin([3, 4, 6, 7, 8]))].shape[0]/df['Permit Type'].isin([3, 4, 6, 7, 8]).shape[0])*100

19.972347913524384

- It can be seen that column Existing Units has 25.9% of its data missing. According to the data dictionary, this column provides existing number of units.
- According to the permit, permit types 3, 8 and 6 have this attribute. There are 24.43% of these permits that have null Existing Units. An explanation for these missing data is error in data entry. A solution for this is to look at the physical permit and manually fill in the data.
- There are 0.99% of the permit types 1, 2, 4, 5 and 7 have Existing Units that is not null. This rows should be dropped.

In [422]:
# Percentage of permit types 3, 8 and 6 that have null Existing Units
(df[(df['Permit Type'].isin([3, 8, 6]))&(df['Existing Units'].isnull())].shape[0]/df['Permit Type'].isin([1, 2, 4, 5, 7]).shape[0])*100

24.491201608848666

In [423]:
# Drop rows of permit types 1, 2, 4, 5 and 7 that have not null Existing Units
df_clean.drop(df_clean[(df_clean['Permit Type'].isin([1, 2, 4, 5, 7]))&(df_clean['Existing Units'].notnull())].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/127393321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean['Permit Type'].isin([1, 2, 4, 5, 7]))&(df_clean['Existing Units'].notnull())].index, inplace=True)


- It can be seen that column Proposed Use has 21.34% of its data missing. According to the data dictionary, this column provides proposed use of the building.
- According to the permit, permit types 3, 8, 4 and 7 have this attribute. However, 20.91% of these permits have null values. A reason for these missing data is error in data entry.
- There are 0.65% of permit types 1, 2, 5 and 6 have Proposed Use that is not null. This rows should be removed   

In [424]:
# Percentage of permit types 3, 8, 4 and 7 that have null Proposed Use
(df[(df["Permit Type"].isin([3, 8, 4, 7])) & (df["Proposed Use"].isnull())].shape[0]/df["Permit Type"].isin([3, 8, 4, 7]).shape[0])*100

20.986928104575163

In [425]:
# Remove rows of permit types 1, 2, 5 and 6 that have Proposed Use that is not null
df_clean.drop(df_clean[(df_clean['Permit Type'].isin([1, 2, 5, 6])) & (df_clean["Proposed Use"].notnull())].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/660126627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean['Permit Type'].isin([1, 2, 5, 6])) & (df_clean["Proposed Use"].notnull())].index, inplace=True)


- It can be seen that column Proposed Units has 25.59% of its data missing. According to the data dictionary, this column provides proposed number of units.
- According to the permit, permit types 4, 5 and 7 don't have this attribute
- 23.47% of permit types 1, 2, 3 and 8 have null Proposed Units. A reason for these missing data is error in data entry. A solution for this is to look at the physical permit and manually fill in the data.

In [426]:
# Percentage of Permit Type 1, 2, 3 and 8 that have null Proposed Units
(df[(df['Permit Type'].isin([1, 2, 3, 8])) & (df['Proposed Units'].isnull())].shape[0]/df['Permit Type'].isin([1, 2, 3, 8]).shape[0])*100

23.53795877325289

- It can be seen that column Plansets has 18.76% of its data missing. According to the data dictionary, this column provides plan representation indicating the general design intent of the foundation.
- A reason for these missing data might be error in data entry. A solution for this is to look at the physical document and manually fill in the data

- It can be seen that column TIDF Compliance has 99.999% of its data missing. According to the data dictionary, this column provides TIDF (a new legal requirement) compliant or not.
- Based on the data dictionary, this column should contains only 2 kinds of values: "Y" and "N". However, there is 1 line contains "P". In conclusion, null values should be replace by "N" and the "P" line should be dropped

In [427]:
# Replace null values in TIDF Compliance with "N"
df_clean['TIDF Compliance'] = df_clean['TIDF Compliance'].fillna(value="N", inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1150433860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['TIDF Compliance'] = df_clean['TIDF Compliance'].fillna(value="N", inplace=True)
/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1150433860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['TIDF Compliance'] = df_clean['TIDF Compliance'].fillna(value="N", inplace=True)


In [428]:
# Drop row that have TIDF Compliance = "P"
df_clean.drop(df_clean[df_clean['TIDF Compliance']=="P"].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/2728994270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[df_clean['TIDF Compliance']=="P"].index, inplace=True)


- It can be seen that column Existing Construction Type has 21.8% of its data missing. According to the data dictionary, this column provides construction type, existing as categories represented numerically
- According to the permit, permit types 1, 2 and 5 don't have this attribute. However, there is one record of permit type 2 that has this attribute. This row should be removed.
- There are 21.03% of permit types 3, 4, 5, 7 and 8 have null Existing Construction Type. The reason for these missing data might be data entry error. One solution is to look at the physical permit and manually enter the data.

In [429]:
# Drop row that have Permit Type 2 and not null Existing Construction Type
df_clean.drop(df_clean[(df_clean['Permit Type'].isin([1, 2, 5])) & (df_clean['Existing Construction Type'].notnull())].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/3926778825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean['Permit Type'].isin([1, 2, 5])) & (df_clean['Existing Construction Type'].notnull())].index, inplace=True)


In [430]:
# Percentage of Permit Type 3, 4, 6, 7 and 8 that have null Existing Construction Type
(df[(df['Permit Type'].isin([3, 4, 6, 7, 8])) & (df['Existing Construction Type'].isnull())].shape[0]/df['Permit Type'].isin([3, 4, 6, 7, 8]).shape[0])*100

21.104575163398692

- It can be seen that column Existing Construction Type Description has 21.8% of its data missing. According to the data dictionary, this column provides description of the above, for example, 
wood or other construction types.
- The percentage of missing data in Existing Construction Type Description matches the percentage of missing data in Existing Construction Type. In addition, the value counts of both attributes are the same.
- In conclusion, when we clean Existing Construction Type, we have simultaneously cleaned Existing Construction Type Description

In [431]:
# Value counts of Existing Construction Type Description
df["Existing Construction Type Description"].value_counts()

wood frame (5)    113350
constr type 1      28072
constr type 3       9663
constr type 2       4068
constr type 4        381
Name: Existing Construction Type Description, dtype: int64

In [432]:
# Value counts of Existing Construction Type
df["Existing Construction Type"].value_counts()

5.0    113350
1.0     28072
3.0      9663
2.0      4068
4.0       381
Name: Existing Construction Type, dtype: int64

- It can be seen that column Proposed Construction Type has 21.7% of its data missing. According to the data dictionary, this column provides information about construction type, proposed, as categories
represented numerically.
- According to the permit, only permit types 3 and 8 have this attribute. However, 19.57% of these permits have null Proposed Construction Type. One explanation for these missing data is error in data entry. A solution for this is to look at the physical permit and manually enter the data.
- 1289 rows or 0.06% of permit types 1, 2, 4, 5, 6 and 7 have Proposed Construction Type that is not null. These rows should be removed.

In [433]:
# Percentage of Permit Types 3 and 8 that have null Proposed Construction Type
(df[(df["Proposed Construction Type"].isnull()) & (df["Permit Type"].isin([3, 8]))].shape[0]/df["Permit Type"].isin([3, 8]).shape[0])*100

19.638511814982405

In [434]:
# Drop rows of Permit Types 1, 2, 4, 5, 6 and 7 that have not null Proposed Construction Type
df_clean.drop(df_clean[(df_clean["Proposed Construction Type"].notnull()) & (df_clean["Permit Type"].isin([1, 2, 4, 5, 6, 7]))].index, inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/2973280872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(df_clean[(df_clean["Proposed Construction Type"].notnull()) & (df_clean["Permit Type"].isin([1, 2, 4, 5, 6, 7]))].index, inplace=True)


- It can be seen that column Proposed Construction Type Description has 21.7% of its data missing. According to the data dictionary, this column provides description of the above
- The percentage of missing data in Proposed Construction Type Description matches the percentage of missing data in Proposed Construction Type. In addition, the value counts of both attributes are the same.
- In conclusion, when we clean Proposed Construction Type, we have simultaneously cleaned Proposed Construction Type Description

In [435]:
# Value counts of Proposed Construction Type
df["Proposed Construction Type"].value_counts()

5.0    114382
1.0     27841
3.0      9360
2.0      3778
4.0       377
Name: Proposed Construction Type, dtype: int64

In [436]:
# Value counts of Proposed Construction Type Description
df["Proposed Construction Type Description"].value_counts()

wood frame (5)    114382
constr type 1      27841
constr type 3       9360
constr type 2       3778
constr type 4        377
Name: Proposed Construction Type Description, dtype: int64

- It can be seen that column Site Permit has 97.3% of its data missing. According to the data dictionary, this column provides information about permit for site.
- Because this column only contains 2 types of values: "nan" and "Y", it is reasonable to repllace "nan" with "N"

In [437]:
# Replace null values of Site Permit with "N"
df_clean["Site Permit"].fillna("N", inplace = True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/2722913401.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Site Permit"].fillna("N", inplace = True)


- It can be seen that the column Neighborhoods - Analysis Boundaries has 0.87% of its data missing. According to the data dictionary, this column provides neighborhood to which the building location belongs to.
- This information has a direct link to the zip code, so we can use the zip code to fill in the missing data.
- There are 10 rows that have missing Neighborhoods - Analysis Boundaries but not null Zipcode.
- We fill in the missing data with the most frequent Neighborhoods - Analysis Boundaries value of the respective Zipcode.
- Because the amount of missing data is not substantial, drop the remaining missing rows

In [438]:
#Rows that have null Neighborhoods - Analysis Boundaries but not null Zipcode
df[(df['Neighborhoods - Analysis Boundaries'].isnull())&(df['Zipcode'].notnull())]

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
73628,201501236544,8,otc alterations permit,01/23/2015,6493,033,7,NaN,Lincoln,Ct,...,5.0,wood frame (5),5.0,wood frame (5),NaN,11.0,NaN,94112.0,"(37.708255336721685, -122.44291501224802)",1368763502711
95269,201508124091,8,otc alterations permit,08/12/2015,6493,033,7,NaN,Lincoln,Ct,...,5.0,wood frame (5),5.0,wood frame (5),NaN,11.0,NaN,94112.0,"(37.708255336721685, -122.44291501224802)",1391862502711
109210,201512165304,8,otc alterations permit,12/16/2015,6499,038,305,NaN,Baltimore,Wy,...,5.0,wood frame (5),5.0,wood frame (5),NaN,11.0,NaN,94112.0,"(37.70826637942377, -122.43729680833803)",1406807205186
111291,201601137013,8,otc alterations permit,01/13/2016,7283,004,207,NaN,Skyline,Bl,...,5.0,wood frame (5),5.0,wood frame (5),NaN,7.0,NaN,94132.0,"(37.71993360931114, -122.4934839342649)",1409101498824
153084,201703282504,8,otc alterations permit,03/28/2017,7283,004,991,NaN,Lake Merced,Bl,...,NaN,NaN,NaN,NaN,NaN,7.0,NaN,94132.0,"(37.71993360931114, -122.4934839342649)",1457580471826
155263,201704244705,8,otc alterations permit,04/24/2017,6432,018,88,NaN,Robinson,Dr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,NaN,NaN,94112.0,"(37.7083761736833, -122.42802141182385)",1460557203010
162451,201707121664,8,otc alterations permit,07/12/2017,7283,004,207,NaN,Skyline,Bl,...,5.0,wood frame (5),5.0,wood frame (5),NaN,7.0,NaN,94132.0,"(37.71993360931114, -122.4934839342649)",1470213498824
175513,201703222121,8,otc alterations permit,03/22/2017,7283,004,1,NaN,Harding,Rd,...,NaN,NaN,NaN,NaN,NaN,7.0,NaN,94132.0,"(37.71993360931114, -122.4934839342649)",1457075504092
177006,201706078660,8,otc alterations permit,06/07/2017,7381,041,143,NaN,Lake Merced Hill St South,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,NaN,NaN,94132.0,"(37.70829338744119, -122.48350972345418)",1466000492201
186364,201710191639,8,otc alterations permit,10/19/2017,6498,012,321,NaN,Polaris,Wy,...,5.0,wood frame (5),5.0,wood frame (5),NaN,NaN,NaN,94112.0,"(37.708277723752225, -122.43921944248987)",1483904205147


In [439]:
#Values of Neighborhoods - Analysis Boundaries that have Zipcode = 94112
df.loc[df['Zipcode']==94112,'Neighborhoods - Analysis Boundaries'].value_counts()

Excelsior                     2724
Outer Mission                 2241
Oceanview/Merced/Ingleside    1651
West of Twin Peaks            1083
Bernal Heights                 192
McLaren Park                     1
Name: Neighborhoods - Analysis Boundaries, dtype: int64

In [440]:
#Create a sub dataframe that only contains rows that have Zipcode = 94112
df_1 = df_clean.loc[df_clean['Zipcode']==94112,]
df_1['Neighborhoods - Analysis Boundaries'].fillna('Excelsior',inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/303826306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Neighborhoods - Analysis Boundaries'].fillna('Excelsior',inplace=True)


In [441]:
#Values of Neighborhoods - Analysis Boundaries that have Zipcode = 94132
df.loc[df['Zipcode']==94132,'Neighborhoods - Analysis Boundaries'].value_counts()

Lakeshore                     1308
Oceanview/Merced/Ingleside     976
Sunset/Parkside                862
West of Twin Peaks             356
Name: Neighborhoods - Analysis Boundaries, dtype: int64

In [442]:
#Create a sub dataframe that only contains rows that have Zipcode = 94112
df_2 = df_clean.loc[df_clean['Zipcode']==94132]
df_2['Neighborhoods - Analysis Boundaries'].fillna('Lakeshore',inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/3162697717.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Neighborhoods - Analysis Boundaries'].fillna('Lakeshore',inplace=True)


In [443]:
#Drop rows that contain Zipcode = 94112 or 94132
df_filtered = df_clean[(df_clean['Zipcode'].isin([94112, 94132]))]
index = df_filtered.index
df_clean.drop(index, inplace=True)

#Concatnate the two sub dataframes with the original dataframe
df_clean = pd.concat([df_1, df_2, df_clean])
df_clean

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/1082133173.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(index, inplace=True)


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
47,M839427,8,otc alterations permit,10/04/2017,6780,034,252,NaN,Cotter,St,...,NaN,NaN,NaN,NaN,N,11.0,Outer Mission,94112.0,"(37.729885737657085, -122.43669590253097)",1482203213002
64,201711154040,3,additions alterations or repairs,11/15/2017,6969,001,5050,NaN,Mission,St,...,NaN,NaN,NaN,NaN,N,11.0,Outer Mission,94112.0,"(37.71772263199437, -122.44060092814006)",1487232215636
97,201602058959,3,additions alterations or repairs,02/05/2016,6404,017,1201,NaN,Geneva,Av,...,5.0,wood frame (5),5.0,wood frame (5),N,11.0,Excelsior,94112.0,"(37.71433687660641, -122.43582765047069)",1411583202293
179,M813548,8,otc alterations permit,07/26/2017,3206,027,43,NaN,Norton,St,...,NaN,NaN,NaN,NaN,N,11.0,Outer Mission,94112.0,"(37.7248056861895, -122.43582950746978)",1471977154849
213,M818747,8,otc alterations permit,08/14/2017,3147,030,4534,NaN,Mission,St,...,NaN,NaN,NaN,NaN,N,11.0,Outer Mission,94112.0,"(37.72545183685408, -122.43458476956798)",1474411246405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,M862628,8,otc alterations permit,12/05/2017,0113,017A,1228,NaN,Montgomery,St,...,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,1489337276729
198896,201712055595,8,otc alterations permit,12/05/2017,0271,014,580,NaN,Bush,St,...,5.0,wood frame (5),5.0,wood frame (5),N,NaN,NaN,NaN,NaN,1489462354993
198897,M863507,8,otc alterations permit,12/06/2017,4318,019,1568,NaN,Indiana,St,...,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,1489539379952
198898,M863747,8,otc alterations permit,12/06/2017,0298,029,795,NaN,Sutter,St,...,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,1489608233656


In [444]:
#Drop the remaining missing values
df_clean.dropna(axis=0, subset='Neighborhoods - Analysis Boundaries', inplace=True)

- It can be seen that column Supervisor District has 0.86% of its data missing. According to the data dictionary, this column provides information about the Supervisor District to which the building location belongs to.
- This information can be based on the column Neighborhoods - Analysis Boundaries to fill in the missing values. However, after cleaning column Neighborhoods - Analysis Boundaries, there are only 3 rows that have null Supervisor District also have not null Neighborhoods - Analysis Boundaries.
    * With regards to when Neighborhoods - Analysis Boundaries equals Excelsior and Zipcode equals 94112, most of the time the Supervisor District equals 11, so it is reasonable to fill in the missing values with 11.
    * With regards to when Neighborhoods - Analysis Boundaries equals Lakeshore and Zipcode equals 94132, all the time the Supervisor District equals 7, so it is reasonable to fill in the missing values with 7.
- In addition, the proportion of missing data is not substanstial, so it is reasonable to drop the remaining missing rows.

In [445]:
#Number of rows that have null Supervisor District but not null Neighborhods - Analysis Boundaries
df_clean[(df_clean['Supervisor District'].isnull())&(df_clean['Neighborhoods - Analysis Boundaries'].notnull())]

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
155263,201704244705,8,otc alterations permit,04/24/2017,6432,018,88,NaN,Robinson,Dr,...,5.0,wood frame (5),5.0,wood frame (5),N,NaN,Excelsior,94112.0,"(37.7083761736833, -122.42802141182385)",1460557203010
186364,201710191639,8,otc alterations permit,10/19/2017,6498,012,321,NaN,Polaris,Wy,...,5.0,wood frame (5),5.0,wood frame (5),N,NaN,Excelsior,94112.0,"(37.708277723752225, -122.43921944248987)",1483904205147
177006,201706078660,8,otc alterations permit,06/07/2017,7381,041,143,NaN,Lake Merced Hill St South,St,...,5.0,wood frame (5),5.0,wood frame (5),N,NaN,Lakeshore,94132.0,"(37.70829338744119, -122.48350972345418)",1466000492201


In [446]:
#Values of Supervisor District when Neighborhoods - Analysis Boundaries equals Excelsior and Zipcode equals 94112
df[(df['Neighborhoods - Analysis Boundaries']=='Excelsior') & (df['Zipcode']==94112)]["Supervisor District"].value_counts()

11.0    2701
9.0       23
Name: Supervisor District, dtype: int64

In [447]:
#Values of Supervisor District when Neighborhoods - Analysis Boundaries equals Lakeshore and Zipcode equals 94132
df[(df['Neighborhoods - Analysis Boundaries']=='Lakeshore') & (df['Zipcode']==94132)]["Supervisor District"].value_counts()

7.0    1308
Name: Supervisor District, dtype: int64

In [448]:
#Create a sub dataframe that only contains rows that have Zipcode = 94112 and Neighborhoods - Analysis Boundaries equals Excelsior
df_1 = df_clean.loc[(df_clean['Zipcode']==94112) & (df_clean['Neighborhoods - Analysis Boundaries']=='Excelsior'),]
df_1['Supervisor District'].fillna(11,inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/167692902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Supervisor District'].fillna(11,inplace=True)


In [449]:
#Create a sub dataframe that only contains rows that have Zipcode = 94132 and Neighborhoods - Analysis Boundaries equals Excelsior
df_2 = df_clean.loc[(df_clean['Zipcode']==94112) & (df_clean['Neighborhoods - Analysis Boundaries']=='Lakeshore'),]
df_2['Supervisor District'].fillna(7,inplace=True)

/var/folders/v0/84n2brq13nl0lv8hgw1lgk180000gn/T/ipykernel_16255/2783921099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Supervisor District'].fillna(7,inplace=True)


In [450]:
#Drop rows that contain Zipcode = 94112 or 94132 and Neighborhoods - Analysis Boundaries equals Excelsior or Lakeshore
df_filtered = df_clean.loc[(df_clean['Zipcode']==94112) & (df_clean['Neighborhoods - Analysis Boundaries']=='Excelsior'),]
index = df_filtered.index
df_clean.drop(index, inplace=True)

df_filtered = df_clean.loc[(df_clean['Zipcode']==94112) & (df_clean['Neighborhoods - Analysis Boundaries']=='Lakeshore'),]
index = df_filtered.index
df_clean.drop(index, inplace=True)

#Concatnate the two sub dataframes with the original dataframe
df_clean = pd.concat([df_1, df_2, df_clean])
df_clean

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
97,201602058959,3,additions alterations or repairs,02/05/2016,6404,017,1201,NaN,Geneva,Av,...,5.0,wood frame (5),5.0,wood frame (5),N,11.0,Excelsior,94112.0,"(37.71433687660641, -122.43582765047069)",1411583202293
384,M852447,8,otc alterations permit,11/06/2017,5890,004A,275,NaN,Maynard,St,...,NaN,NaN,NaN,NaN,N,11.0,Excelsior,94112.0,"(37.729703788032396, -122.42436027184064)",1486137190413
468,M869007,8,otc alterations permit,12/20/2017,6079,015A,486,NaN,Edinburg,St,...,NaN,NaN,NaN,NaN,N,11.0,Excelsior,94112.0,"(37.7215758009115, -122.43209195778869)",1491320195468
473,M869487,8,otc alterations permit,12/21/2017,6020,007,350,NaN,Vienna,St,...,NaN,NaN,NaN,NaN,N,11.0,Excelsior,94112.0,"(37.72289424614288, -122.42891347413111)",1491491194302
752,201301047252,8,otc alterations permit,01/04/2013,6409,010,965,NaN,Geneva,Av,...,2.0,constr type 2,2.0,constr type 2,N,11.0,Excelsior,94112.0,"(37.71611785405651, -122.43971056334294)",1292388202430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198880,M893387,8,otc alterations permit,02/23/2018,1841,025D,1442,NaN,Funston,Av,...,NaN,NaN,NaN,NaN,N,7.0,Inner Sunset,94122.0,"(37.76117190796411, -122.47007880106999)",1498337116842
198881,M893407,8,otc alterations permit,02/23/2018,0018,004,2300,NaN,Stockton,St,...,NaN,NaN,NaN,NaN,N,3.0,North Beach,94133.0,"(37.80747140583783, -122.41024025753224)",1498338347099
198883,M893427,8,otc alterations permit,02/23/2018,0315,021,377,NaN,Geary,St,...,NaN,NaN,NaN,NaN,N,3.0,Tenderloin,94102.0,"(37.78696971667916, -122.40936701144197)",149834069380
198884,M893447,8,otc alterations permit,02/23/2018,3504,030,1699,NaN,Market,St,...,NaN,NaN,NaN,NaN,N,6.0,Mission,94103.0,"(37.77217130976155, -122.42193576313322)",1498341156230


In [451]:
#Drop rows that have missing Supervisor District
df_clean.dropna(subset='Supervisor District', inplace=True)

- In the orriginal dataset, 0.86% of the cells in column Location is missing. However after cleaning all of the previous columns, the number of missing cells in column Location is zero

In [454]:
#Number of missing cells in column Location in the final dataset
df_clean['Location'].isnull().sum()

0

In [455]:
#Export the clean dataset
df_clean.to_csv("Building_Permits_clean.csv", index=False)